In [1]:
import tensorflow as tf

import argparse
import numpy as np
from keras.callbacks import CSVLogger, ModelCheckpoint

from models.vaes_β import MSAVAE
from utils.io import load_data
from utils.data_loaders import one_hot_generator
from models.vaes_β import MSAVAE
from utils.io import load_data
from utils.data_loaders import one_hot_generator
import gc
import os
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# Define training parameters


Using TensorFlow backend.


In [2]:
batch_size = 500

In [3]:
seed = 15737
n_epochs = 1
verbose = 1
save_all_epochs = False
seed = np.random.seed(seed)

In [4]:
aa_letters = ['-', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K',
              'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [5]:
def to_one_hot2(seqlist, alphabet=aa_letters):
    aa_key = {l: i for i, l in enumerate(alphabet)}
    if type(seqlist) == str:
        return seq_to_one_hot2(seqlist, aa_key)
    else:
        encoded_seqs = []
        for prot in seqlist:
            encoded_seqs.append(seq_to_one_hot2(prot, aa_key))
        return np.stack(encoded_seqs)

In [6]:

def seq_to_one_hot2(sequence, aa_key):
    arr = np.zeros((len(sequence),len(aa_key)))
    for j, c in enumerate(sequence):
        arr[j, aa_key[c]] = 1
    return arr

In [7]:
evaluate_list = []

In [8]:
model = MSAVAE(original_dim=253, latent_dim=10)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
jogjpr
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
dcu
Learning rate  0.001
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Protein VAE initialized !


In [9]:
weights_file = "output/defaultparameter/msavae_BLAT_train_val_8vs2.h5"
model.load_weights(weights_file)

file =  output/defaultparameter/msavae_BLAT_train_val_8vs2.h5
Weights loaded !


In [10]:
for i in range(5):
    _,all_msa_seqs = load_data('data/BLAT_data/BLAT_data' + str(i+1)+ '.fasta', one_hot=False)
    _, all_val_msa_seqs = load_data('data/BLAT_data/BLAT_data' + str(i+1)+ '.fasta', one_hot=False)
    msa_seqs = []
    val_msa_seqs = []
    for j in range(100):
        for k in range(500):
            msa_seqs.append(all_msa_seqs[k+500*j])
            val_msa_seqs.append(all_val_msa_seqs[k+500*j])
        train_gen = to_one_hot2(msa_seqs)
        val_gen = to_one_hot2(msa_seqs)
        evaluate_list.append(model.VAE.evaluate(train_gen,val_gen,batch_size=500))
        msa_seqs.clear()
        val_msa_seqs.clear()

500/500 [==============================] - 0s 152us/step


500/500 [==============================] - 0s 152us/step


500/500 [==============================] - 0s 160us/step


500/500 [==============================] - 0s 152us/step


In [11]:
len(evaluate_list)

500

In [12]:
f = open('BLAT_500mutant_train_val_stop_val_acc_8vs2.csv', 'w',encoding='shift_jis')

In [13]:
for i in range(len(evaluate_list)):
    f.write("変異体"+str(i)+","+str(-evaluate_list[i][0])+"\n")

In [14]:
f.close()

In [10]:
evaluate_list = []

In [12]:
evaluate_list

[[18.818012237548828,
  0.994560182094574,
  0.9945612549781799,
  7.745937347412109,
  11.072074890136719,
  18.81801414489746]]

In [11]:
evaluate_list.append(model.VAE.evaluate(train_gen,val_gen,batch_size=500))

500/500 [==============================] - 0s 642us/step


In [12]:
print("test loss, test acc:", evaluate)

test loss, test acc: [18.690829071044924, 0.9946879482269287, 0.9946877450942994, 7.618754951477051, 11.072073974609374, 18.690829071044924]


In [10]:
learning = model.VAE.fit_generator(generator=train_gen,steps_per_epoch=len(msa_seqs) // batch_size,verbose=verbose,
                                epochs=n_epochs,validation_data=val_gen,
                                validation_steps=len(val_msa_seqs) // batch_size,callbacks=callbacks)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [4]:
for j in range(len(all_msa_seqs)//500):
        for k in range(500):
            msa_seqs.append(all_msa_seqs[k+500*j])
            val_msa_seqs.append(all_val_msa_seqs[k+500*j])
        train_gen = one_hot_generator(msa_seqs, padding=None)
        val_gen = one_hot_generator(val_msa_seqs, padding=None)
# Define model
        model_kwargs=None
        print('Building model')
        model = MSAVAE(original_dim=253, latent_dim=10)
        if model_kwargs is None:
            model_kwargs = {}
        else:
            with open(model_kwargs, 'rb') as p:
                model_kwargs = pickle.load(p)
        # (Optionally) define callbacks
        model = MSAVAE(**model_kwargs)
        weights_file = "output/defaultparameter/msavae_BLAT.h5"
        model.load_weights(weights_file)
        callbacks=[CSVLogger('output/logs/msavae'+str(j)+'.csv')]
        if save_all_epochs:
            callbacks.append(ModelCheckpoint('output/weights/msavae_BLAT'+'.{epoch:02d}-{luxa_errors_mean:.2f}.hdf5',save_best_only=False, verbose=1))
        print('Training model')
        learning = model.VAE.fit_generator(generator=train_gen,steps_per_epoch=len(msa_seqs) // batch_size,verbose=verbose,\
                                        epochs=n_epochs,validation_data=val_gen,\
                                        validation_steps=len(val_msa_seqs) // batch_size,callbacks=callbacks)
                       
        print("番号 : ",j+i*100)
        if not save_all_epochs:
            model.save_weights('output/weights/msavae_BLAT.h5') 
        
        
        print("活性評価")
        print(learning.history['vae_loss'][-1])
        print(-learning.history['vae_loss'][-1])
       # print("+活性評価 = ",-learning.history['vae_loss'][-1],"- = "-learning.history['vae_loss'][-1])
        if j == 0 and i == 0:
            f.write("親配列,"+str(-learning.history['vae_loss'][-1])+"\n")
        else:
            f.write("変異体"+str(j+i*100)+","+str(-learning.history['vae_loss'][-1])+"\n")
        msa_seqs.clear()
        val_msa_seqs.clear()




z_mean =  Tensor("dense_3/BiasAdd:0", shape=(?, 10), dtype=float32)  : z_var =  Tensor("dense_4/Softplus:0", shape=(?, 10), dtype=float32)
E =  <keras.engine.training.Model object at 0x000001967BF26278>
Learning rate  0.001
Protein VAE initialized !
Weights loaded !


AttributeError: 'MSAVAE' object has no attribute 'parameters'